In [2]:
import requests
import json
import os

url = "https://core.hivelocity.net/api/v1/products/displays/www/fast"

try:
    # response = requests.get(url)
    if reponse.status_code == 200:
        data_json = json.loads(response.json())
        all_locations = data_json['ALL_LOCATIONS']
        path = os.getcwd() + "/data.json"
        with open(path, 'w') as file:
            json.dump(all_locations, file, indent=4)
        
except Exception as e:
    print(f"Error: {str(e)}")

Error: name 'reponse' is not defined


In [37]:
def count_words_numbers(obj):
    word_count = 0
    number_count = 0

    if isinstance(obj, dict):
        for value in obj.values():
            words, numbers = count_words_numbers(value)
            word_count += words
            number_count += numbers
    elif isinstance(obj, list):
        for item in obj:
            words, numbers = count_words_numbers(item)
            word_count += words
            number_count += numbers
    elif isinstance(obj, str):
        word_count += len(obj.split())
    elif isinstance(obj, (int, float)):
        number_count += 1

    return word_count, number_count

In [41]:
import json
from bs4 import BeautifulSoup

path = os.getcwd() + "/data.json"
with open(path, 'r') as file:
    data = json.load(file)
store_url = "https://store.hivelocity.net/product/"
def clean_data(data):
    """
    Cleans the input data by normalizing boolean values, removing HTML tags,
    copying 'cores' from processor_text to display_text, and removing processor_text.

    :param data: Dictionary containing the data to be cleaned.
    :return: Cleaned data as a dictionary.
    """

    # Function to remove HTML tags
    def remove_html_tags(text):
        return BeautifulSoup(text, "html.parser").get_text()

    if 'processor_text' in data and 'display_text' in data:
        if 'cores' in data['processor_text']:
            cores_text = remove_html_tags(data['processor_text']['cores'])
            data['display_text']['cores'] = cores_text.replace('(', '').replace(')', '')
            
        del data['processor_text']
        data['description'] = data.pop('display_text')
        
    if 'id' in data:
        data['product_id'] = data.pop('id')
        data['product_store_url'] = f"{store_url}{data['product_id']}/customize"
        
    if 'category' in data and 'id' in data['category']:
        data['category']['category_id'] = data['category'].pop('id')
        
    for key, value in data.items():
        if isinstance(value, bool):
            data[key] = bool(value) 
        elif isinstance(value, dict):
            data[key] = clean_data(value)
            
    if 'original_price' in data and 'price' in data and data['price'] < data['original_price']:
        discount = data['original_price'] - data['price']
        data['discount'] = {
            'has_discount': True,
            'discount_amount': discount,
            'discount_percentage': round((discount / data['original_price']) * 100, 2)
        }
    return data

data_ = [clean_data(item) for item in data if item['stock'] > 0]
num_token = count_words_numbers(data_)
print(num_token)
print(len(data_))

(3353, 1936)
146
[{"plan_id": 496, "stock": 27, "sale": true, "new": false, "original_price": 85.0, "category": {"name": "Single Processor", "priority": 0, "slug": "single-processor", "category_id": 1}, "price": 40.0, "sps": true, "vps": false, "description": {"name": "E3-1270 v3 3.5GHz Haswell ", "cpu": "E3-1270 v3", "memory": "32GB", "drive": "480GB SSD", "bandwidth": "20TB / 1Gbps", "gpu": "None", "cores": "4 cores/8 threads"}, "product_id": 2337, "product_store_url": "https://store.hivelocity.net/product/2337/customize", "discount": {"has_discount": true, "discount_amount": 45.0, "discount_percentage": 52.94}}]
